# Create a nested dictionary in Python, 10 levels deep, varying on number of leading spaces

I have a file that looks like this:

```
Object=Project
  Comment=TestComment
  Version=1.0.0
  Flags=$01
PrintSettings=Settings
  Lang=English
  PrintFirstPage=True
  Admin=True
    User=Author
EndTags
```

And I want a nested dictionary that encapsulates the data varying on the number of leading spaces in each line (0, level 0; 2, level 1; 4, level 2; ... ; 20 level 10), so the result is:
```
defaultdict(<class 'dict'>, {'Object': 'Project', 'Project': {'Comment': 'TestComment', 'Version': '1.0.0', 'Flags': '$01'}, 'PrintSettings': 'Settings', 'Settings': {'Lang': 'English', 'PrintFirstPage': 'True', 'Admin': 'True', 'True': {'User': 'Author'}}})
```
I have this piece of code that reproduces the result given, but only functions properly with 2 levels of depth:


In [1]:
string = """Object=Project
  Comment=TestComment
  Version=1.0.0
  Flags=$01
PrintSettings=Settings
  Lang=English
  PrintFirstPage=True
  Admin=True
    User=Author
EndTags"""

In [1]:
from collections import defaultdict

lines_infile = []
numleadingspaces = []
dictionary_fromfile = defaultdict(dict)

for line in str.splitlines(string):
    line = line.rstrip()
    lines_infile.append(line)
    numleadingspaces.append(len(line)-len(line.lstrip()))

for i in range(len(lines_infile)):

    if '=' in lines_infile[i]:
        if numleadingspaces[i] == 0:
            level0_key, level0_value = lines_infile[i].split('=')
            level0_key = level0_key.lstrip()
            dictionary_fromfile[level0_key] = level0_value
            last_level0_key, last_level0_value = level0_key, level0_value
        
        elif numleadingspaces[i] == 2:
            level0key_forlevel2 = last_level0_value
            level2_key, level2_value = lines_infile[i].split('=')
            level2_key = level2_key.lstrip()
            dictionary_fromfile[level0key_forlevel2][level2_key] = level2_value
            last_level2_key, last_level2_value = level2_key, level2_value
        
        elif numleadingspaces[i] == 4:
            level2key_forlevel4 = last_level2_value
            level4_key, level4_value = lines_infile[i].split('=')
            level4_key = level4_key.lstrip()
            dictionary_fromfile[level0key_forlevel2][level2key_forlevel4] = {}
            dictionary_fromfile[level0key_forlevel2][level2key_forlevel4][level4_key] = level4_value
            
display(dictionary_fromfile)

NameError: name 'string' is not defined

# My version

In [4]:
result = defaultdict(dict)
temp = result # Create a view of the dictionary
level = []
prev_indent = 0
prev_value = ""

for line in str.splitlines(string):
    if "=" in line:
        
        indent = round(int(len(line)-len(line.lstrip()))/2)
        line = line.strip()

        key, value = str.split(line, "=")

        if indent > prev_indent:
            prev_indent = indent
            temp[prev_value] = {} # Create nested dict
            temp = temp[prev_value] # Change the view of the dict
            
        elif indent < prev_indent:
            prev_indent = indent
            level = level[0:indent]
            temp = result # Change the view of the dict
            for l in level:
                temp = temp[l]

        prev_value = value
            
        temp[key] = value

display(result)

defaultdict(dict,
            {'Object': 'Project',
             'Project': {'Comment': 'TestComment',
              'Version': '1.0.0',
              'Flags': '$01'},
             'PrintSettings': 'Settings',
             'Settings': {'Lang': 'English',
              'PrintFirstPage': 'True',
              'Admin': 'True',
              'True': {'User': 'Author'}}})